In [178]:
import pandas as pd
from scipy import stats
import numpy as np
import xlrd

In [68]:
data=pd.read_table('/Users/yiyezhang/Documents/Data/NYPData/HF/data/order_2012_2018.txt',sep='\t',error_bad_lines=False,names=["clientvisitguid","clientguid","userguid", "enterrole", "chartguid","createdwhen","enteredwhen","name","ordersetname","typecode","orderguid", "requestedtime","frequencycode","duration"])

/Users/yiyezhang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


In [7]:
#data=pd.concat([data18, data1217], axis=0)

In [64]:
print(len(data))

3901845


In [71]:
data['enterrole'].unique()

array(['Nurse', 'Resident', 'PA', 'NP', 'Physician', 'VerifyChemoTherapy',
       'Medical Resident', 'Fellow', 'Attending Physician',
       'Registered Nurse', 'Nurse Anesthetist'], dtype=object)

In [70]:
data=data[data['enterrole'].isin(['PA', 'Resident', 'Nurse', 'NP', 'Physician', 'VerifyChemoTherapy',
       'Fellow', 'Medical Resident', 'Attending Physician',
       'Registered Nurse', 'Nurse Anesthetist'])]

In [72]:
len(data)

3049494

In [73]:
data['userguid']=data['userguid'].astype(int)
data['clientvisitguid']=data['clientvisitguid'].astype(int)

In [74]:
data['createdwhen'] = data['createdwhen'].astype('datetime64[h]')  #change to m/s for minutes and seconds

In [75]:
len(data['userguid'].unique())

7113

In [76]:
data['year'] = pd.DatetimeIndex(data['createdwhen']).year

In [77]:
data.groupby(['year'])[['clientguid']].nunique()

,clientguid
year,
2012,3
2013,236
2014,1548
2015,1908
2016,2333
2017,1667
2018,1529


In [78]:
visit_user=data.groupby(['userguid','clientvisitguid'])[['name']].count()

In [79]:
visit_user=visit_user.reset_index()

In [80]:
visit_user.columns.values

array(['userguid', 'clientvisitguid', 'name'], dtype=object)

In [81]:
len(visit_user)

285307

In [82]:
visit_user_grp_by_user_mean=visit_user.groupby(['userguid'])[['name']].sum()

In [83]:
mean_user_orders=visit_user_grp_by_user_mean.reset_index()

In [365]:
#164 orders
#26 visits
#164/26 = 6.3 OS on average per visit 

In [84]:
len(mean_user_orders)

7113

In [85]:
len(mean_user_orders['userguid'].unique())

7113

In [38]:
# visit_user_grp_by_user_mean=visit_user_grp_by_user_mean[visit_user_grp_by_user_mean.Name >=2]

In [89]:
mean_user_orders=mean_user_orders[(np.abs(stats.zscore(mean_user_orders['name']) < 3))]

In [90]:
mean_user_orders['name'].mean()

264.89802440441605

In [91]:
mean_user_orders['name'].std()

576.9735671822017

In [92]:
mean_user_orders['name'].min()

1

In [93]:
mean_user_orders.loc[mean_user_orders['name'].idxmax()]

userguid    9000062289801190
name                    3728
Name: 4624, dtype: int64

In [94]:
mean_user_orders.to_csv(path+"order_2012_2018_ordercount.csv")

In [95]:
data_OS=data[data['userguid'].isin(mean_user_orders['userguid'])]

In [96]:
data_OS=data[data['ordersetname'].notna()]

In [97]:
len(data_OS)

1384925

In [98]:
len(data_OS['clientvisitguid'].unique())

11676

In [99]:
len(data_OS['userguid'].unique())

2918

In [100]:
user_role=data[['userguid','enterrole']].drop_duplicates()

In [101]:
user_role.to_csv("2012_2018_user_role.csv")

In [102]:
user_OS=data_OS.groupby(['userguid'])[['ordersetname']].count()

In [103]:
user_OS=user_OS.reset_index()

In [104]:
user_OS['ordersetname'].mean()

474.61446196024673

In [105]:
user_OS['ordersetname'].std()

769.9023470682192

In [106]:
user_OS.to_csv("order_2012_2018_OScount.csv")

In [107]:
len(data_OS)

1384925

In [109]:
data_OS['userguid'].nunique()

2918

In [179]:
path = '/Users/yiyezhang/Documents/Data/NYPData/HF/data/'

In [110]:
diag = pd.read_csv(path + 'DIAG_2012_2018.txt',sep='\t',names=["clientguid","clientvisitguid","createdwhen","typecode","shortname","icd10code","icd9code","clientguid","userguid"])

In [111]:
print(len(diag))

200370


In [113]:
data.head()

,clientvisitguid,clientguid,userguid,enterrole,chartguid,createdwhen,enteredwhen,name,ordersetname,typecode,orderguid,requestedtime,frequencycode,duration,year
0,9000745799000270,9e+15,9000002162801190,Nurse,9000745799900170,2015-05-25 22:00:00,2015-05-25 22:34:32.977,13 Lead EKG (Peds),ED Order Set - Nursing Pre-Assessment Order Set,Diagnostic,9.026509e+15,STAT,NaN,0.0,2015
1,9000745799000270,9e+15,9000041735501190,Resident,9000745799900170,2015-05-25 22:00:00,2015-05-25 22:49:25.760,Chest Pa And Lateral,NaN,Diagnostic,9.026509e+15,Routine,NaN,0.0,2015
2,9000745799000270,9e+15,9000041735501190,Resident,9000745799900170,2015-05-25 22:00:00,2015-05-25 22:55:17.023,Ibuprofen Oral,NaN,Medication,9.026509e+15,Routine,Once,180.0,2015
3,9000745799000270,9e+15,9000041735501190,Resident,9000745799900170,2015-05-25 22:00:00,2015-05-25 22:55:17.023,Cyclobenzaprine Oral,NaN,Medication,9.026509e+15,Routine,Once,180.0,2015
4,9000745799000270,9e+15,9000041735501190,Resident,9000745799900170,2015-05-26 00:00:00,2015-05-26 00:56:57.277,oxyCODONE/Acetaminophen 5/325 mg Tab (Percocet),NaN,Medication,9.026510e+15,Routine,Once,7.0,2015


In [114]:
diag['createdwhen'] = pd.to_datetime(diag['createdwhen'])

In [115]:
diag['createdwhen'] = diag['createdwhen'].astype('datetime64[h]')

In [116]:
diag['year'] = pd.DatetimeIndex(diag['createdwhen']).year
diag.groupby(['year'])[['clientvisitguid']].nunique()

,clientvisitguid
year,
2012,1
2013,241
2014,1876
2015,2405
2016,3043
2017,3207
2018,3328


In [174]:
demographics = pd.read_csv(path + 'demographics_2012_2018.csv',sep='\t',names=["clientguid","ethnicity1","ethnicity2","YOB","language","gender","race","mariatalstatus"])

In [175]:
len(demographics)

8617

In [123]:
len(data[data['clientvisitguid'].isin(diag['clientvisitguid'])]['clientguid'].unique())

7124

In [59]:
#diag2.to_csv("diag_acuteHF_2018.csv", index=False)

In [120]:
diag['icd10top3']=diag['icd10code'].str.split('.').str[0]

In [122]:
datadic={}
for row in diag.itertuples():
    datadic[row.clientvisitguid]={}
    datadic[row.clientvisitguid]['ED']=[]
    datadic[row.clientvisitguid]['Discharge']=[]
    datadic[row.clientvisitguid]['Acute']=[]
    datadic[row.clientvisitguid]['Admitting']=[]
    datadic[row.clientvisitguid]['Principal']=[]
    datadic[row.clientvisitguid]['Billing']=[]
    datadic[row.clientvisitguid]['Initial']=[]
    datadic[row.clientvisitguid]['Last']=[]
    

In [125]:
for row in diag.itertuples():
    if row.typecode=='ED Diagnosis':
        datadic[row.clientvisitguid]['ED'].append(row.icd10code)
        datadic[row.clientvisitguid]['Initial'].append(row.icd10code)
    elif row.typecode=='Acute Dx':
        datadic[row.clientvisitguid]['Acute'].append(row.icd10code)
        datadic[row.clientvisitguid]['Initial'].append(row.icd10code)
    elif row.typecode=='Admitting Dx':
        datadic[row.clientvisitguid]['Admitting'].append(row.icd10code)
        datadic[row.clientvisitguid]['Initial'].append(row.icd10code)
    elif row.typecode=='Principal Dx':
        datadic[row.clientvisitguid]['Principal'].append(row.icd10code)
        datadic[row.clientvisitguid]['Last'].append(row.icd10code)
    elif row.typecode=='Billing Dx':
        datadic[row.clientvisitguid]['Billing'].append(row.icd10code)
        datadic[row.clientvisitguid]['Last'].append(row.icd10code)
    elif row.typecode=='Discharge Dx':
        datadic[row.clientvisitguid]['Discharge'].append(row.icd10code)
        datadic[row.clientvisitguid]['Last'].append(row.icd10code)

In [233]:
EDDC=[]
for pid in datadic:
    for i in datadic[pid]['ED']:
        for j in datadic[pid]['Discharge']:
            print(i,j)
            if i==j and i in ('R57.0', 'I09.81', 'I11.0', 'I13.0', 'I13.2', 'I50', 'I50.1', 'I50.2', 'I50.20', 'I50.21', 'I50.22', 'I50.23', 'I50.3', 'I50.30', 'I50.31', 'I50.32', 'I50.33', 'I50.4', 'I50.40', 'I50.41', 'I50.42', 'I50.43', 'I50.9', 'I50.22', 'I50.32', 'I50.42', 'I50.2', 'I50.20', 'I50.21', 'I50.23', 'I50.3', 'I50.30', 'I50.31', 'I50.33', 'I50.4', 'I50.40', 'I50.41', 'I50.43', 'I50.3', 'I50.30', 'I50.31', 'I50.32', 'I50.33', 'I50.1', 'I50.2', 'I50.20', 'I50.21', 'I50.22', 'I50.23', 'I50.4', 'I50.40', 'I50.41', 'I50.42', 'I50.43', 'I50.9'):
                EDDC.append(pid)

R33.9 I13.10
R33.9 N17.9
R07.9 N39.0
R07.9 R70.0
R51 N39.0
R51 R70.0
M79.1 N39.0
M79.1 R70.0
R53.1 K92.2
R62.7 K92.2
R06.00 I50.33
R06.00 E87.70
R06.00 I50.9
I50.9 E87.70
I50.9 I50.9
E87.70 E87.70
E87.70 I50.9
I48.91 E87.70
I48.91 I50.9
R07.9 I50.9
R07.9 J96.00
R07.9 I10
R07.9 N18.9
R06.02 I50.9
R06.02 J96.00
R06.02 I10
R06.02 N18.9
R53.1 A41.9
R07.9 I26.99
M79.89 I26.99
M25.539 M11.9
M25.579 M11.9
R41.82 R06.00
R41.82 C50.919
R41.82 R07.9
R41.82 I48.91
R41.82 I50.9
R41.82 N39.0
R41.82 I49.5
R41.82 D61.810
R41.82 D69.6
R41.82 D64.9
R41.82 D70.9
A41.9 R06.00
A41.9 C50.919
A41.9 R07.9
A41.9 I48.91
A41.9 I50.9
A41.9 N39.0
A41.9 I49.5
A41.9 D61.810
A41.9 D69.6
A41.9 D64.9
A41.9 D70.9
R07.9 K59.00
R07.9 R30.0
R07.9 Z87.09
R07.9 I25.10
Z95.0 I50.9
R00.2 I50.9
R14.0 E05.80
E73.9 E05.80
I50.9 E05.80
N17.9 E05.80
R74.8 I50.9
R42 I50.9
R42 I50.9
S79.912A F03.90
S72.90XA F03.90
R00.0 E23.2
R00.0 E87.0
R00.0 A41.9
A41.9 E23.2
A41.9 E87.0
A41.9 A41.9
R53.81 E23.2
R53.81 E87.0
R53.81 A41.9
D89.9 E23

N28.9 R31.9
N28.9 C64.9
N28.9 I50.43
N28.9 J90
N28.9 J90
I50.23 R31.9
I50.23 C64.9
I50.23 I50.43
I50.23 J90
I50.23 J90
I50.23 R31.9
I50.23 C64.9
I50.23 I50.43
I50.23 J90
I50.23 J90
J90 R31.9
J90 C64.9
J90 I50.43
J90 J90
J90 J90
N02.9 R31.9
N02.9 C64.9
N02.9 I50.43
N02.9 J90
N02.9 J90
R06.02 I50.23
I50.23 I50.23
R06.89 I50.9
M79.89 I50.9
I50.9 I50.9
I50.9 I50.9
R07.9 R07.89
R07.9 I50.30
R07.9 J45.901
R07.9 B34.8
R07.9 E11.9
R07.9 E87.70
R07.9 J02.9
R07.9 G47.33
R05 R07.89
R05 I50.30
R05 J45.901
R05 B34.8
R05 E11.9
R05 E87.70
R05 J02.9
R05 G47.33
I48.91 B33.8
I48.91 I50.20
I48.91 J96.00
C80.1 C79.9
C80.1 J81.1
C80.1 I50.23
C80.1 J96.00
R06.02 K85.90
R06.02 K81.0
R79.89 K85.90
R79.89 K81.0
R06.02 I50.21
R06.02 I48.91
R06.02 N17.9
R41.0 R57.0
R41.82 R57.0
A41.9 R57.0
I48.91 I31.9
R06.02 I34.0
R06.02 J44.1
R06.02 R73.9
R06.02 I48.92
R06.02 I50.23
R06.02 I34.0
R06.02 N17.9
R06.02 I16.0
R06.02 I50.33
R06.02 N17.9
R06.02 I16.1
J81.0 I16.0
J81.0 I50.33
J81.0 N17.9
J81.0 I16.1
I63.9 I63.9
I63.9 

I95.9 I50.9
I95.9 B18.1
R06.02 D69.6
R06.02 D64.9
R06.02 N18.9
R06.02 I50.9
R06.02 B18.1
R00.1 D69.6
R00.1 D64.9
R00.1 N18.9
R00.1 I50.9
R00.1 B18.1
R53.1 D69.6
R53.1 D64.9
R53.1 N18.9
R53.1 I50.9
R53.1 B18.1
I99.8 D69.6
I99.8 D64.9
I99.8 N18.9
I99.8 I50.9
I99.8 B18.1
J90 D69.6
J90 D64.9
J90 N18.9
J90 I50.9
J90 B18.1
W19.XXXA T14.8XXA
N39.0 T14.8XXA
T14.91XA R09.02
T14.91XA R45.1
T14.91XA N30.80
T14.91XA F03.90
T14.91XA F03.90
R45.1 R09.02
R45.1 R45.1
R45.1 N30.80
R45.1 F03.90
R45.1 F03.90
R06.00 I24.9
I50.9 I24.9
I24.9 I24.9
R53.1 R53.1
R53.1 I48.91
R53.1 N18.9
R53.1 I50.9
R53.1 R74.0
R29.6 I50.9
R29.6 R74.0
R60.1 I50.9
R60.1 R74.0
R06.02 C34.90
R07.9 E87.1
R07.9 I31.3
R07.9 R11.2
R07.9 I50.22
R07.9 R41.82
R07.9 R50.9
R06.02 E87.1
R06.02 I31.3
R06.02 R11.2
R06.02 I50.22
R06.02 R41.82
R06.02 R50.9
R53.1 I50.30
R53.1 I48.91
R53.1 D50.9
R53.1 N17.9
R53.1 R55
R53.1 I48.91
R09.02 I50.30
R09.02 I48.91
R09.02 D50.9
R09.02 N17.9
R09.02 R55
R09.02 I48.91
R41.89 I50.30
R41.89 I48.91
R41.89 D50.

R22.0 I50.9
R60.0 I50.9
I10 I50.9
I50.9 I50.9
R06.02 J84.10
R06.02 I50.9
M54.9 J96.01
R06.02 N18.5
R06.02 E11.9
R06.02 N18.6
I50.9 N18.5
I50.9 E11.9
I50.9 N18.6
N18.6 N18.5
N18.6 E11.9
N18.6 N18.6
R06.02 I50.9
R05 R74.8
R05 J18.9
R05 A41.9
J18.9 R74.8
J18.9 J18.9
J18.9 A41.9
R22.42 C49.20
Z85.831 C49.20
L03.116 C49.20
C78.00 C49.20
R06.00 R09.02
R06.00 E87.70
J44.1 R09.02
J44.1 E87.70
J44.9 R09.02
J44.9 E87.70
R06.89 R09.02
R06.89 E87.70
S09.90XA I48.91
R55 I48.91
I48.91 I48.91
R68.83 I50.9
R68.83 E87.70
R41.82 I50.9
R41.82 E87.70
R53.83 I50.9
R53.83 E87.70
R05 I50.9
R05 E87.70
R06.02 J98.09
R06.02 Z43.0
R06.02 Z93.1
R06.02 E11.9
R06.02 N39.0
R06.02 N17.9
R06.02 E87.5
R53.1 J98.09
R53.1 Z43.0
R53.1 Z93.1
R53.1 E11.9
R53.1 N39.0
R53.1 N17.9
R53.1 E87.5
R06.00 J11.1
R06.00 I21.4
R06.00 J96.20
R06.00 I51.9
R74.8 J11.1
R74.8 I21.4
R74.8 J96.20
R74.8 I51.9
S09.90XA I50.9
S09.90XA R29.6
R56.9 I50.9
R56.9 R29.6
W19.XXXA I50.9
W19.XXXA R29.6
S01.01XA I50.9
S01.01XA R29.6
I50.9 I50.9
I50.9 R29.

R06.02 N18.9
R06.02 M48.02
R01.1 Z95.2
R01.1 I35.0
R01.1 I50.30
R01.1 N18.9
R01.1 M48.02
S09.90XA R55
R55 R55
Z43.1 I21.3
Z43.1 J38.00
Z43.1 J95.851
Z43.1 I25.5
R07.9 I50.23
I48.91 I50.23
R06.02 I50.23
R06.00 D86.0
R06.00 I63.9
R06.00 J96.00
R79.89 D86.0
R79.89 I63.9
R79.89 J96.00
R79.89 D86.0
R79.89 I63.9
R79.89 J96.00
R79.89 D86.0
R79.89 I63.9
R79.89 J96.00
E87.2 D86.0
E87.2 I63.9
E87.2 J96.00
E87.2 D86.0
E87.2 I63.9
E87.2 J96.00
D86.0 D86.0
D86.0 I63.9
D86.0 J96.00
R07.9 I50.9
R07.9 J44.1
I50.9 I50.9
I50.9 J44.1
J44.1 I50.9
J44.1 J44.1
R06.00 I50.9
R06.00 J44.1
I48.91 G62.9
I48.91 E87.70
I48.91 R79.1
T79.8XXA I50.9
T79.8XXA I48.91
T79.8XXA G62.9
T79.8XXA F31.9
T79.8XXA F11.20
T79.8XXA I71.2
R06.02 I50.9
R06.02 I48.91
R06.02 G62.9
R06.02 F31.9
R06.02 F11.20
R06.02 I71.2
M79.89 I50.9
M79.89 I48.91
M79.89 G62.9
M79.89 F31.9
M79.89 F11.20
M79.89 I71.2
R07.9 I22.2
R00.0 I48.91
I48.91 I48.91
R06.00 R09.02
R06.00 J96.01
R06.00 J44.1
R06.00 I50.20
R06.00 E87.5
I50.9 R09.02
I50.9 J96.01
I50.

R42 I34.0
R42 I50.23
R42 R07.89
R42 I47.9
R10.9 K83.9
R10.9 R11.2
R06.02 I50.9
R06.02 I50.9
R05 R94.31
J18.9 R94.31
I16.0 R94.31
R10.9 K86.1
R42 J44.1
R42 I50.30
R06.02 J44.1
R06.02 I50.30
M79.89 J44.1
M79.89 I50.30
R06.02 I50.9
I50.9 I50.9
I50.9 I50.9
N17.9 I50.9
I21.4 I50.9
R10.13 I48.91
R10.13 I50.30
R07.9 I48.91
R07.9 I50.30
D61.818 R04.0
D61.818 I50.9
R04.0 R04.0
R04.0 I50.9
D69.6 R04.0
D69.6 I50.9
I82.409 I82.409
R05 I50.23
R05 E11.9
R05 I25.10
R05 N17.9
R05 J18.9
R06.02 I50.23
R06.02 E11.9
R06.02 I25.10
R06.02 N17.9
R06.02 J18.9
I50.9 I50.23
I50.9 E11.9
I50.9 I25.10
I50.9 N17.9
I50.9 J18.9
R10.9 E83.51
R10.9 E87.1
R10.9 W19.XXXA
R10.9 K50.00
R10.9 K92.2
R10.9 E83.51
W19.XXXA E83.51
W19.XXXA E87.1
W19.XXXA W19.XXXA
W19.XXXA K50.00
W19.XXXA K92.2
W19.XXXA E83.51
I50.1 E83.51
I50.1 E87.1
I50.1 W19.XXXA
I50.1 K50.00
I50.1 K92.2
I50.1 E83.51
I10 E83.51
I10 E87.1
I10 W19.XXXA
I10 K50.00
I10 K92.2
I10 E83.51
W19.XXXA N39.0
W19.XXXA I50.30
W19.XXXA E87.70
W19.XXXA I50.9
W19.XXXA I50.9
W

In [234]:
EDDC=list(set(EDDC)) 

In [235]:
DC=[]
for pid in datadic:
    for j in datadic[pid]['Discharge']:
        if j in ('R57.0', 'I09.81', 'I11.0', 'I13.0', 'I13.2', 'I50', 'I50.1', 'I50.2', 'I50.20', 'I50.21', 'I50.22', 'I50.23', 'I50.3', 'I50.30', 'I50.31', 'I50.32', 'I50.33', 'I50.4', 'I50.40', 'I50.41', 'I50.42', 'I50.43', 'I50.9', 'I50.22', 'I50.32', 'I50.42', 'I50.2', 'I50.20', 'I50.21', 'I50.23', 'I50.3', 'I50.30', 'I50.31', 'I50.33', 'I50.4', 'I50.40', 'I50.41', 'I50.43', 'I50.3', 'I50.30', 'I50.31', 'I50.32', 'I50.33', 'I50.1', 'I50.2', 'I50.20', 'I50.21', 'I50.22', 'I50.23', 'I50.4', 'I50.40', 'I50.41', 'I50.42', 'I50.43', 'I50.9'):
            DC.append(pid)

In [236]:
DC=list(set(DC)) 

In [126]:
initlast=[]
for pid in datadic:
    for i in datadic[pid]['Initial']:
        for j in datadic[pid]['Last']:
            if i==j and i in ('R57.0', 'I09.81', 'I11.0', 'I13.0', 'I13.2', 'I50', 'I50.1', 'I50.2', 'I50.20', 'I50.21', 'I50.22', 'I50.23', 'I50.3', 'I50.30', 'I50.31', 'I50.32', 'I50.33', 'I50.4', 'I50.40', 'I50.41', 'I50.42', 'I50.43', 'I50.9', 'I50.22', 'I50.32', 'I50.42', 'I50.2', 'I50.20', 'I50.21', 'I50.23', 'I50.3', 'I50.30', 'I50.31', 'I50.33', 'I50.4', 'I50.40', 'I50.41', 'I50.43', 'I50.3', 'I50.30', 'I50.31', 'I50.32', 'I50.33', 'I50.1', 'I50.2', 'I50.20', 'I50.21', 'I50.22', 'I50.23', 'I50.4', 'I50.40', 'I50.41', 'I50.42', 'I50.43', 'I50.9'):
                initlast.append(pid)

In [127]:
initlast=list(set(initlast)) 

In [128]:
len(initlast)

2033

In [ ]:
#EDDC=initiallast from here now on

In [129]:
diag_EDDC=diag[diag['clientvisitguid'].isin(initlast)]

In [130]:
len(diag_EDDC)

34284

In [131]:
diag_EDDC.to_csv(path+"diag_EDDC_2012_2018.csv", index=False)

In [132]:
order_EDDC=data[data['clientvisitguid'].isin(initlast)]

In [133]:
order_EDDC['userguid'].nunique()

4030

In [134]:
order_EDDC.to_csv(path+"order_EDDC_2012_2018.csv", index=False)

In [135]:
demographics_EDDC=demographics[demographics['clientguid'].isin(order_EDDC['clientguid'])]

In [136]:
demographics_EDDC.to_csv(path+"demographics_EDDC_2012_2018.csv", index=False)

In [194]:
clientvisit=pd.read_csv(path + 'demographics_2012_2018_jupiter.csv',sep=',',header=0)

In [201]:
clientvisit['HospitalService'].unique()

array(['MED', nan, 'SUR', 'THO', 'PSY', 'MIC', 'FRA', 'NES', 'REH', 'NEU',
       'IAC', 'HOS', 'PED', 'URO', 'MOT', 'BUR', 'OBS', 'NIC', 'POS',
       'EYE', 'ANE', 'GYN', 'LOA', 'ENT', 'NBN'], dtype=object)

In [196]:
clientvisit.iloc[0:3]

,DischargeDCDesc,DischargeDtm,AdmitDtm,ClientVisitGUID,ClientGUID,HospitalService
0,DISCHARGE HOME WITH HM HLTH SERV,2014-02-20 11:19:00.000,2014-02-13 19:43:00.000,9000581681600270,9000007484300200,MED
1,PATIENT EXPIRED,2018-01-05 14:30:00.000,2018-01-02 17:54:00.000,9001106617100270,9000030306500200,MED
2,DISCHARGE HOME WITH HM HLTH SERV,2015-06-25 19:00:00.000,2015-06-18 14:35:00.000,9000755088100270,9000195661000200,MED


In [197]:
clientvisit['AdmitDtm'] = clientvisit['AdmitDtm'].astype('datetime64[h]')

In [198]:
clientvisit['DischargeDtm'] = clientvisit['DischargeDtm'].astype('datetime64[h]')

In [199]:
clientvisit=clientvisit[~clientvisit['DischargeDtm'].isna()]

In [200]:
clientvisit_EDDC=clientvisit[clientvisit['ClientVisitGUID'].isin(initlast)]

In [145]:
clientvisit_EDDC.to_csv(path+"visit_EDDC_2012_2018.csv", index=False)

In [54]:
#order_EDDC=order_EDDC[order_EDDC['userguid'].isin(userguid_EDDC_18_89)]

In [163]:
clientvisit_EDDC['clientvisitguid'].nunique()

1994

In [147]:
order_EDDC_OSonly=order_EDDC[order_EDDC['ordersetname'].notna()]

In [148]:
order_EDDC_OSonly.shape

(202471, 15)

In [149]:
user_order_EDDC_OSonly=order_EDDC_OSonly.groupby(['userguid'])[['ordersetname']].count()

In [150]:
user_order_EDDC_OSonly=user_order_EDDC_OSonly.reset_index()

In [151]:
user_order_EDDC=order_EDDC.groupby(['userguid'])[['name']].count()

In [152]:
user_order_EDDC=user_order_EDDC.reset_index()

In [153]:
user_order_EDDC.to_csv(path+"user_order_EDDC_2012_2018.csv", index=False)

In [154]:
user_order_EDDC_OSonly.to_csv(path+"user_order_EDDC_OSonly_2012_2018.csv", index=False)

In [155]:
order_EDDC['typecode'].unique()

array(['Other', 'Diagnostic', 'Medication', nan], dtype=object)

In [156]:
user_quartile=pd.read_excel('EDDC_order_2012_2018_analysis.xlsx', sheetname='MDOnly_1order_peryear')

/Users/yiyezhang/anaconda3/lib/python3.7/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [157]:
OS1Q=user_quartile[user_quartile['Q1']==1]

In [158]:
OS3Q=user_quartile[user_quartile['Q4']==1]

In [159]:
OS3Q.shape

(382, 9)

In [74]:
data_1Q=order_EDDC[order_EDDC['userguid'].isin(OS1Q['userguid'])]

In [75]:
data_3Q=order_EDDC[order_EDDC['userguid'].isin(OS3Q['userguid'])]

In [76]:
data_1Q.to_csv(path+"EDDC_18_89_order_acuteHF_2018_1Q.csv")

In [77]:
data_3Q.to_csv(path+"EDDC_18_89_order_acuteHF_2018_3Q.csv")

In [211]:
diag_3Q=diag[diag['userguid'].isin(OS3Q['userguid'])]

In [212]:
diag_1Q=diag[diag['userguid'].isin(OS1Q['userguid'])]

In [214]:
diag_1Q['icd10top3']=diag_1Q['icd10code'].str.split('.').str[0]

/Users/yiyezhang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [215]:
diag_3Q['icd10top3']=diag_3Q['icd10code'].str.split('.').str[0]

/Users/yiyezhang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [216]:
diag_1Q.to_csv(path+"EDDC_diag_acuteHF_2018_1Q.csv")

In [217]:
diag_3Q.to_csv(path+"EDDC_diag_acuteHF_2018_3Q.csv")

In [359]:
diag_3Q.iloc[0:10]

,clientvisitguid,createdwhen,typecode,shortname,icd10code,icd9code,userguid,icd10top3
105,9001274504100270,2018-12-07 13:00:00,Principal Dx,Volume overload,E87.70,276.69,9000025622301190,E87
276,9001283668100270,2018-12-27 22:00:00,Chronic Dx,AKI (acute kidney injury),N17.9,584.9,9000030070401190,N17
284,9001283668100270,2018-12-29 22:00:00,Chronic Dx,OSA (obstructive sleep apnea),G47.33,327.23,9000030070401190,G47
428,9001166443200270,2018-05-02 13:00:00,ED Diagnosis,New onset of congestive heart failure,I50.9,428.0,9000034230901190,I50
513,9001216566000270,2018-08-14 19:00:00,Acute Dx,"Ventricular tachycardia, monomorphic",I47.2,427.1,9000030070401190,I47
590,9001184509100270,2018-06-06 17:00:00,Acute Dx,Near syncope,R55,780.2,9000007325101190,R55
591,9001184509100270,2018-06-06 18:00:00,ED Diagnosis,Chest pain,R07.9,786.50,9000007325101190,R07
600,9001248036700270,2018-10-15 22:00:00,Acute Dx,Weakness,R53.1,780.79,9000081830001190,R53
1105,9001216566000270,2018-08-16 10:00:00,Chronic Dx,"Ventricular tachycardia, monomorphic",I47.2,427.1,9000030070401190,I47
1126,9001133593600270,2018-02-26 00:00:00,ED Diagnosis,"Atrial tachycardia, paroxysmal",I47.1,427.0,9000166186001190,I47


In [361]:
order_EDDC['name'].nunique()

2438

In [362]:
data.iloc[0:9]

,clientvisitguid,clientguid,userguid,enterrole,chartguid,createdwhen,enteredwhen,name,ordersetname,typecode,orderguid,requestedtime,frequencycode,duration
0,9001193205200270,9.000001e+15,9000083986801190,PA,9001191403800170,2018-06-25 08:00:00,2018-06-25 08:42:37.157,ECG/12 Lead-POC ED,NaN,Other,9.048799e+15,Routine,Once,0.0
1,9001193205200270,9.000001e+15,9000083986801190,PA,9001191403800170,2018-06-25 08:00:00,2018-06-25 08:43:59.290,"IV, Peripheral Access",ED Order Set - A General Comprehensive Order Set,Other,9.048799e+15,EMER,<Continuous>,0.0
2,9001193205200270,9.000001e+15,9000083986801190,PA,9001191403800170,2018-06-25 08:00:00,2018-06-25 08:43:59.290,Activated Partial Thromboplastin Time,ED Order Set - A General Comprehensive Order Set,Diagnostic,9.048799e+15,EMER,NaN,0.0
3,9001193205200270,9.000001e+15,9000083986801190,PA,9001191403800170,2018-06-25 08:00:00,2018-06-25 08:43:59.290,Basic Metabolic Panel,ED Order Set - A General Comprehensive Order Set,Diagnostic,9.048799e+15,EMER,NaN,0.0
4,9001193205200270,9.000001e+15,9000083986801190,PA,9001191403800170,2018-06-25 08:00:00,2018-06-25 08:43:59.290,Complete Blood Count with Differential,ED Order Set - A General Comprehensive Order Set,Diagnostic,9.048799e+15,EMER,NaN,0.0
5,9001193205200270,9.000001e+15,9000083986801190,PA,9001191403800170,2018-06-25 08:00:00,2018-06-25 08:43:59.290,Lipase,ED Order Set - A General Comprehensive Order Set,Diagnostic,9.048799e+15,Now,NaN,0.0
6,9001193205200270,9.000001e+15,9000083986801190,PA,9001191403800170,2018-06-25 08:00:00,2018-06-25 08:43:59.290,Liver Function Panel,ED Order Set - A General Comprehensive Order Set,Diagnostic,9.048799e+15,Now,NaN,0.0
7,9001193205200270,9.000001e+15,9000083986801190,PA,9001191403800170,2018-06-25 08:00:00,2018-06-25 08:43:59.290,Magnesium,ED Order Set - A General Comprehensive Order Set,Diagnostic,9.048799e+15,EMER,NaN,0.0
8,9001193205200270,9.000001e+15,9000083986801190,PA,9001191403800170,2018-06-25 08:00:00,2018-06-25 08:43:59.290,PT/INR,ED Order Set - A General Comprehensive Order Set,Diagnostic,9.048799e+15,EMER,NaN,0.0


In [369]:
HFonly=[]
for pid in datadic:
    if pid in initlast and len(datadic[pid]['Last'])==1:
        print(pid,datadic[pid]['Last'])
        HFonly.append(pid)

9001251915900270 ['I50.33']
9001216906000270 ['I50.9']
9001121093200270 ['I50.9']
9001173168800270 ['I50.9']
9001211775200270 ['I50.30']
9001246400900270 ['I50.30']
9001220441100270 ['I50.23']
9001174426900270 ['I50.9']
9001247254900270 ['I50.9']
9001109270200270 ['I50.31']
9001176608700270 ['I50.23']
9001241780400270 ['I50.9']
9001234306600270 ['I50.9']
9001206891600270 ['I50.23']
9001179229100270 ['I50.9']
9001265609400270 ['I50.9']
9001148298400270 ['I50.30']
9001282931300270 ['I50.9']
9001246979700270 ['I50.23']
9001236154600270 ['I50.43']
9001232695300270 ['I50.33']
9001283883400270 ['I50.33']
9001225771000270 ['I50.23']
9001180467800270 ['I50.23']
9001235742900270 ['I50.9']
9001140573400270 ['I50.9']
9001262036300270 ['I50.9']
9001277843700270 ['I50.9']
9001191425900270 ['I50.23']
9001170806400270 ['I50.23']
9001143998000270 ['I50.9']
9001115258800270 ['I50.23']
9001251701600270 ['I50.9']
9001189761100270 ['I50.33']
9001172050300270 ['I50.23']
9001213409300270 ['I50.9']
900117412

In [1]:
len(HFonly)

NameError: name 'HFonly' is not defined

In [50]:
userguid_EDDC_18_89=[0, 9000028999901190, 9000025620701190, 9000011771101190, 9000022087901190, 9000017223901190, 3000001301119001, 9000118474801190, 9000028209201190, 9000002916401190, 9000174052401190, 9000017559601190, 9000216010801190, 9000025831301190, 9000053888901190, 9000060493701190, 9000206720901190, 9000043267801190, 9000193795801190, 9000119607001190, 9000134122201190, 9000041450201190, 9000212023001190, 9000119018201190, 9000118483501190, 9000166176301190, 9000015341101190, 9000001952301190, 9000190496301190, 9000215328301190, 9000022332801190, 9000172502401190, 9000211158401190, 9000119359701190, 9000116646101190, 9000036031701190, 9000049241301190, 9000185996501190, 4000001216119001, 9000202690601190, 9000211164201190, 9000181289001190, 9000065167401190, 9000137820201190, 9000089487401190, 9000182876201190, 9000131167101190, 9000183007101190, 9000191403901190, 9000047096701190, 9000018527101190, 9000201823101190, 9000119598801190, 9000166165201190, 9000211554001190, 9000221230801190, 9000017429201190, 9000001995301190, 9000177406501190, 9000166168101190, 9000179275301190, 9000052455801190, 9000062030201190, 9000078158201190, 9000127182201190, 9000217575801190, 9000166171001190, 9000002228601190, 9000166173901190, 9000062826701190, 9000215658701190, 9000229149901190, 9000002897101190, 9000228893901190, 9000022055201190, 9000029319201190, 9000166176801190, 9000059706401190, 9000002029601190, 9000025607201190, 9000041837601190, 9000208263201190, 9000233095201190, 9000167843701190, 9000227952501190, 9000211158901190, 9000001188119001, 9000185689801190, 9000117875401190, 9000041638601190, 9000022925001190, 9000046041801190, 9000033344201190, 9000030070301190, 9000002908701190, 9000166185501190, 9000170793501190, 9000019548701190, 9000020601201190, 9000093202801190, 9000083986801190, 9000060255601190, 9000077791601190, 9000143970501190, 9000024751301190, 9000060975301190, 9000002172101190, 9000052092101190, 9000139186201190, 9000139391001190, 9000106981401190, 9000166168601190, 9000161128301190, 9000030158701190, 9000042088301190, 9000040987501190, 9000201601901190, 9000211560301190, 9000095876801190, 9000029726401190, 9000166174401190, 9000179921601190, 9000186321601190, 9000024913601190, 9000132100801190, 9000105118401190, 9000092241601190, 9000176442901190, 9000059706901190, 9000041735701190, 9000124628501190, 9000166177301190, 9000021998101190, 9000145441301190, 9000133870101190, 9000118205801190, 9000021796201190, 9000112676201190, 9000166180201190, 9000086948201190, 9000059709801190, 9000166157501190, 9000117875901190, 9000157786301190, 9000023770301190, 9000119002301190, 9000211162301190, 9000041639101190, 9000211546301190, 9000118493201190, 9000170794001190, 9000166186001190, 9000072592401190, 9000001936401190, 9000211165201190, 9000166160401190, 9000024694801190, 9000170976101190, 9000166163301190, 9000030150501190, 9000030304101190, 9000168134501190, 9000059718501190, 9000186617701190, 9000166188901190, 9000031840101190, 9000205587301190, 4000001274119001, 9000017430201190, 9000119599801190, 9000002172601190, 9000013129401190, 9000220847801190, 9000072345101190, 9000060492301190, 9000017945101190, 9000026085901190, 9000032139801190, 6000001331119001, 9000166172001190, 9000211560801190, 9000101071201190, 9000158415201190, 9000166223201190, 9000118760801190, 9000217807201190, 9000180485301190, 9000020434101190, 9000023762101190, 9000034670601190, 9000166177801190, 9000119534601190, 9000033416201190, 9000002903901190, 9000117233501190, 9000166180701190, 9000023051101190, 9000041519201190, 9000184052401190, 9000115879601190, 9000179444401190, 9000211162801190, 9000133415601190, 9000221633201190, 9000211165701190, 9000026615301190, 9000166160901190, 9000062020101190, 9000118493701190, 9000171818501190, 9000029943301190, 9000211242501190, 9000119594501190, 9000002167301190, 9000166189401190, 9000166163801190, 9000062947501190, 9000032253101190, 9000207897601190, 9000184601601190, 9000176844801190, 9000214604801190, 9000070300501190, 9000001999701190, 9000119606101190, 9000060495701190, 1104119001, 9000119609001190, 9000211154601190, 9000030162601190, 9000166175401190, 9000119532201190, 9000061781501190, 9000118997501190, 9000166178301190, 9000118488401190, 9000120587601190, 9000004210001190, 9000127243601190, 9000116645201190, 9000002187601190, 9000004312401190, 9000211160401190, 9000036030801190, 9000045812901190, 9000204123301190, 9000166158501190, 9000118235301190, 9000039592101190, 9000119540901190, 9000213646501190, 3000001218119001, 9000120593401190, 9000028254201190, 9000060254201190, 9000119595001190, 9000029611001190, 9000212318201190, 9000211166201190, 9000001940301190, 9000001914701190, 9000187258701190, 9000021114701190, 9000060973901190, 9000023623501190, 9000194836301190, 9000001275119001, 9000103728801190, 9000025648801190, 9000018480801190, 9000042109601190, 9000028157601190, 9000108234401190, 9000166170101190, 9000119603701190, 9000204877301190, 9000024349001190, 9000047209801190, 9000024195401190, 9000062288201190, 9000034230601190, 9000131766601190, 9000055453001190, 9000026374301190, 9000175340701190, 9000166175901190, 9000151865501190, 6000001261119001, 8000001133119001, 9000024162401190, 9000166178801190, 9000118998001190, 9000016879601190, 9000138428401190, 9000040636401190, 9000211158001190, 9000053592901190, 9000041765701190, 9000118488901190, 9000022284101190, 9000211160901190, 9000027762501190, 9000010050201190, 9000202690201190, 9000212776601190, 9000183669401190, 9000001935001190, 9000128475801190, 9000048808601190, 9000019013101190, 9000174046701190, 9000211166701190, 9000060254701190, 9000166161901190, 9000002705901190, 9000002117101190, 4000001119119001, 9000097505601190, 9000021115201190, 9000120110401190, 9000034094401190, 9000166190401190, 9000119163201190, 9000118474901190, 9000118756501190, 9000078154901190, 9000138775701190, 9000004094101190, 9000039092201190, 9000118196201190, 9000166170601190, 9000060493801190, 9000023629801190, 9000059879401190, 9000111619901190, 9000160976701190, 9000024349501190, 9000062851901190, 9000166173501190, 9000034231101190, 9000029341501190, 9000152938301190, 9000105808701190, 9000002205501190, 9000119533201190, 9000155373201190, 9000118483601190, 9000215328401190, 9000211158501190, 9000052156901190, 9000208163301190, 9000023638501190, 9000002057701190, 9000004361701190, 1000001034119001, 9000211545401190, 9000023615801190, 9000029941901190, 9000211164301190, 9000020418701190, 9000048809101190, 5000001219119001, 1000001091119001, 9000067858401190, 9000112018401190, 9000029944801190, 9000061253601190, 9000187564001190, 9000041567201190, 9000046968801190, 9000043359201190, 9000166165301190, 9000077051701190, 9000002171701190, 9000028846901190, 9000042110601190, 9000118757001190, 9000023806601190, 9000084760201190, 9000051557001190, 9000026289801190, 9000150526601190, 9000183937501190, 9000025908701190, 9000062004701190, 9000085403101190, 9000028263901190, 9000062030301190, 9000023246301190, 9000002922801190, 9000002103601190, 9000106244401190, 9000179921201190, 9000227281201190, 9000229150001190, 9000077242501190, 9000041453701190, 9000118484101190, 9000041735301190, 9000006484101190, 9000002007001190, 9000062832601190, 9000172503001190, 9000055690201190, 9000059709401190, 9000034083801190, 9000150256601190, 9000211235801190, 9000211568601190, 9000117875501190, 9000001958701190, 9000116646701190, 9000211160601190, 1063119001, 9000001319119001, 7000001191119001, 9000211164801190, 9000030070401190, 9000082243201190, 9000166185601190, 9000039260801190, 9000029609601190, 9000230902401190, 9000117599701190, 9000060255701190, 9000166188501190, 9000034066901190, 9000166162901190, 9000112018901190, 9000002220501190, 9000051935701190, 9000166165801190, 9000025391401190, 9000023625001190, 9000034069801190, 9000211554601190, 9000139186301190, 9000062027901190, 9000004453501190, 9000066379901190, 9000028927101190, 9000002075601190, 9000022504401190, 9000189288401190, 9000211563301190, 9000002180901190, 9000095876901190, 9000029316901190, 9000186321701190, 9000126340901190, 9000041735801190, 9000171758201190, 9000223854201190, 9000042225101190, 9000059709901190, 9000023665101190, 9000075197901190, 9000025892301190, 9000146263501190, 9000017853901190, 9000211162401190, 9000181594401190, 9000166183201190, 9000118183201190, 9000156660001190, 9000062938401190, 9000023616801190, 9000022388001190, 9000211546401190, 9000132826401190, 9000183517301190, 9000029430901190, 9000028202101190, 5000001220119001, 9000030304201190, 9000118982601190, 9000125536201190, 9000029997001190, 9000167008201190, 9000002019101190, 9000030076701190, 9000025622301190, 9000020758301190, 9000043878001190, 9000119602801190, 9000030156401190, 9000166169201190, 9000060492401190, 9000210585201190, 9000003535301190, 9000190492101190, 9000130946501190, 9000095106501190, 9000211560901190, 9000217807301190, 9000229839301190, 6000001078119001, 9000062290201190, 9000166175001190, 9000119608601190, 9000025195801190, 9000211154201190, 9000179922201190, 9000026888301190, 9000166177901190, 9000104072301190, 9000046574701190, 9000127243201190, 9000166180801190, 9000022846401190, 9000124222401190, 9000022155201190, 9000028097301190, 9000034369301190, 9000060711701190, 9000167796501190, 9000042228501190, 9000166158101190, 9000211162901190, 9000211546901190, 9000002039401190, 9000170794601190, 9000118493801190, 9000028999101190, 9000026336701190, 9000035373501190, 9000098736401190, 9000119600401190, 9000218237201190, 9000211558501190, 9000166169701190, 9000028032101190, 9000207897701190, 9000107545701190, 9000170265701190, 1000001178119001, 6000001178119001, 9000042115001190, 9000025142201190, 9000060495801190, 9000001951501190, 9000119609101190, 9000025608801190, 9000002594401190, 9000042223201190, 9000166178401190, 9000184943201190, 9000117871201190, 9000043042401190, 9000211160501190, 9000127243701190, 9000118488501190, 9000127602101190, 9000206373301190, 9000001985801190, 9000166158601190, 9000164264201190, 9000211547401190, 9000029608201190, 9000212318301190, 9000091614301190, 9000060254301190, 9000029611101190, 9000142689201190, 9000029434801190, 9000055214001190, 9000166164401190, 9000187258801190, 9000123133701190, 9000042314501190, 9000052784901190, 9000172055301190, 9000023268101190, 9000139722501190, 2000001278119001, 9000166170201190, 9000092237401190, 9000033255001190, 9000176666201190, 9000060493401190, 9000062848601190, 9000017434201190, 9000109415001190, 9000031309401190, 9000211559001190, 9000007325101190, 9000119606701190, 9000180486401190, 9000062291201190, 9000118483201190, 9000211158101190, 9000061782101190, 9000030038101190, 9000062012501190, 9000117871701190, 9000118998101190, 9000002031701190, 9000082569301190, 9000211567701190, 6000001136119001, 9000081829801190, 9000037490601190, 9000061349801190, 9000029938601190, 9000166181801190, 9000118489001190, 9000211161001190, 9000211545001190, 9000211570601190, 9000105563901190, 9000172738301190, 9000202690301190, 9000165595901190, 9000100315901190, 1321119001, 9000187563601190, 9000166187601190, 9000002091601190, 9000123461201190, 9000060254801190, 9000119237201190, 9000187256401190, 9000047096401190, 9000001963601190, 9000211166801190, 9000166162001190, 9000058209701190, 9000184212901190, 9000001992101190, 9000211553701190, 9000071470501190, 9000002071801190, 9000166167801190, 9000042110201190, 9000143460601190, 9000093694201190, 9000118475001190, 9000002100301190, 9000119604301190, 9000073780301190, 9000221620301190, 9000033844301190, 9000211152801190, 9000004023201190, 9000166173601190, 9000173318901190, 9000155373301190, 9000141677301190, 9000118483701190, 9000166176501190, 9000062291701190, 9000146080501190, 9000023635701190, 9000150253301190, 9000034080501190, 8000001236119001, 9000062013001190, 9000022333001190, 9000042224201190, 9000034032201190, 9000211158601190, 9000052157001190, 9000201254301190, 9000002111901190, 9000116646301190, 9000119539101190, 9000001037119001, 1000001293119001, 9000166185201190, 9000211164401190, 9000116114501190, 9000123461701190, 9000166188101190, 9000060255301190, 9000117599301190, 9000021010501190, 9000166162501190, 9000221612101190, 9000020629401190, 9000063816601190, 9000118757101190, 9000179275501190, 9000030616301190, 9000180683501190, 9000001995501190, 9000062027501190, 9000026136301190, 5000001279119001, 9000176411201190, 9000161435201190, 9000034280001190, 9000218395201190, 9000001796501190, 9000034078101190, 9000101303701190, 9000166174101190, 9000031620501190, 9000080132501190, 9000041735401190, 9000035309801190, 9000047469801190, 9000141677801190, 9000166177001190, 9000060500201190, 9000023738601190, 9000119533801190, 9000030113001190, 9000018647101190, 9000211568701190, 9000062832701190, 9000038128701190, 9000183434301190, 9000001904701190, 9000211159101190, 9000211235901190, 9000121021501190, 9000166157201190, 9000041638801190, 9000116646801190, 9000039491301190, 9000045891301190, 9000062019301190, 9000230902501190, 9000117599801190, 9000214470201190, 9000154591801190, 9000166188601190, 9000060255801190, 9000230470201190, 9000075337101190, 9000211554701190, 9000117884301190, 9000005730701190, 9000080661901190, 9000007625101190, 9000166165901190, 9000026303001190, 9000186751201190, 9000004530401190, 9000029567201190, 9000023602401190, 9000219928801190, 9000190491701190, 9000024578101190, 9000089294901190, 9000002101301190, 1000001308119001, 2000001052119001, 1308119001, 9000062289801190, 9000132101001190, 9000029317001190, 9000023736201190, 9000039352201190, 9000002181001190, 9000119608201190, 9000026324701190, 9000166177501190, 9000005870301190, 9000041761501190, 9000023867101190, 9000047470301190, 9000001723101190, 5000001237119001, 9000032983601190, 9000041457201190, 9000025662001190, 9000222456401190, 9000036900401190, 9000166234501190, 9000101056901190, 9000002906501190, 9000211162501190, 9000020621701190, 3000001166119001, 9000122768601190, 9000072592601190, 9000143274201190, 9000024695001190, 9000053725401190, 9000211165401190, 9000028202201190, 9000118493401190, 9000211242201190, 9000219152601190, 9000060256301190, 9000071392301190, 9000177990701190, 9000117856301190, 9000049453101190, 9000017965101190, 9000211552301190, 9000116653101190, 9000166189101190, 9000032304001190, 9000020630401190, 9000166166401190, 9000183011201190, 9000060976001190, 9000160281301190, 9000166169301190, 9000106982101190, 9000004684501190, 9000010854101190, 7000001024119001, 9000176412201190, 9000060495401190, 9000029673001190, 9000118761001190, 9000083074601190, 9000166223401190, 9000031900201190, 9000003535401190, 9000166175101190, 9000178923901190, 9000211563901190, 9000166178001190, 9000084718801190, 9000094011601190, 9000055076901190, 9000127243301190, 9000166180901190, 3000001266119001, 9000211163001190, 9000118158201190, 9000026382201190, 9000166183801190, 9000174734201190, 9000219150201190, 9000211547001190, 9000025028301190, 9000197802701190, 9000170794701190, 9000013713101190, 9000118493901190, 9000173329101190, 9000214775501190, 9000187258401190, 9000025722401190, 9000031661601190, 9000134112801190, 9000051911201190, 9000032045601190, 9000087216501190, 9000166141301190, 9000062848201190, 9000209894601190, 9000031155401190, 9000002611401190, 9000197581001190, 9000222131401190, 9000003510301190, 9000003126301190, 9000031798301190, 9000138550301190, 9000125519901190, 9000025836401190, 9000025785201190, 9000041452401190, 9000119481201190, 9000112774001190, 9000117871301190, 9000104072901190, 9000030037701190, 9000211157701190, 9000202968601190, 9000001983001190, 9000119538201190, 9000025202201190, 9000116645401190, 9000127243801190, 9000118488601190, 9000166181401190, 9000001906201190, 9000041765401190, 9000029941101190, 9000025281901190, 9000166184301190, 9000039387501190, 9000029608301190, 9000045659501190, 1295119001, 9000046481601190, 9000027947201190, 9000060254401190, 9000100728001190, 9000031505601190, 9000137720001190, 9000076203201190, 9000039057601190, 9000212318401190, 9000097505301190, 9000166164501190, 9000187258901190, 9000117601301190, 9000031841301190, 9000166190101190, 9000045742101190, 9000185031701190, 9000192279401190, 9000171108201190, 9000166141801190, 9000118474601190, 9000029309801190, 9000166167401190, 9000119524201190, 9000029799101190, 9000119603901190, 9000028544701190, 9000060493501190, 9000222131901190, 9000152013501190, 1000001025119001, 9000002000401190, 9000166173201190, 9000193795601190, 9000077290001190, 9000042115601190, 9000151865701190, 9000118483301190, 9000166176101190, 9000017388901190, 9000002054501190, 9000002233701190, 9000062291301190, 8000001082119001, 9000003798201190, 9000173910201190, 9000044886201190, 9000002569401190, 9000117871801190, 9000062012601190, 9000211158201190, 9000022332601190, 9000031062201190, 9000023308301190, 9000166181901190, 9000203993101190, 9000211161101190, 9000001932301190, 9000005797901190, 9000041765901190, 9000118489101190, 8000001139119001, 9000211548001190, 9000091458401190, 9000165596001190, 9000029941601190, 9000030069601190, 9000179164001190, 9000018447201190, 9000043151201190, 9000135132001190, 9000211166901190, 9000204536501190, 9000166162101190, 9000034398901190, 9000166187701190, 9000061253301190, 9000048043701190, 9000060254901190, 9000212318901190, 9000166190601190, 9000119598601190, 9000211553801190, 9000054475101190, 9000118475101190, 9000202010801190, 9000033844401190, 9000036967601190, 9000002228401190, 9000204801201190, 9000193796101190, 9000041476101190, 9000156087301190, 9000020714501190, 9000062826501190, 9000166173701190, 6000001310119001, 9000038842201190, 9000160954201190, 9000156141401190, 9000025607001190, 9000176442201190, 9000118483801190, 9000033008301190, 9000004669101190, 9000211235501190, 9000211158701190, 9000125961901190, 9000138429101190, 9000042227201190, 9000117875201190, 9000132134401190, 9000023923201190, 9000104076801190, 9000185689601190, 9000132979201190, 9000046835201190, 9000211545601190, 9000166182401190, 9000030172501190, 9000182876501190, 9000166185301190, 9000170793301190, 9000062021801190, 9000074924201190, 9000046969001190, 9000166188201190, 9000060255401190, 9000117599401190, 9000149599401190, 9000181292201190, 9000171513001190, 9000053010601190, 9000212322301190, 9000006114301190, 9000062843901190, 9000169263101190, 9000032840701190, 1000001097119001, 9000029310801190, 9000001995601190, 9000205157201190, 9000166168401190, 9000042110801190, 9000152600401190, 9000037707601190, 9000041573201190, 9000167192401190, 9000233541201190, 9000008219301190, 9000072347301190, 9000040116901190, 9000028264101190, 9000027163301190, 9000002049701190, 9000002052601190, 9000101303801190, 9000023479801190, 9000094622201190, 9000049540601190, 9000166174201190, 9000041399801190, 9000227281401190, 9000002846201190, 9000042221901190, 9000041735501190, 9000211156301190, 9000041761101190, 9000166177101190, 9000060500301190, 9000059706701190, 9000024839501190, 9000034081101190, 9000016983201190, 9000160983201190, 9000188557301190, 9000166157301190, 9000116646901190, 9000117875701190, 9000064678901190, 9000016909301190, 9000211546101190, 9000079450101190, 7000001268119001, 9000166185801190, 9000166160201190, 9000172944201190, 9000230902601190, 9000128300701190, 9000060255901190, 9000166163101190, 9000023647901190, 9000007625201190, 9000060975601190, 9000166166001190, 9000020604401190, 9000022908401190, 9000182322501190, 9000117528901190, 9000186751301190, 9000213759301190, 9000047823001190, 9000042114201190, 9000034306201190, 9000060981401190, 9000020968601190, 9000179970201190, 9000211563501190, 9000192696301190, 9000088504301190, 9000166174701190, 9000030417901190, 9000145902401190, 9000031393601190, 9000002058901190, 9000029937301190, 9000166180501190, 9000075198101190, 9000032138901190, 9000116388501190, 9000117233301190, 9000042228201190, 9000001933801190, 9000166157801190, 9000116647401190, 9000211162601190, 9000171818301190, 9000118493501190, 9000030813501190, 9000211165501190, 9000170794301190, 9000166160701190, 9000172637501190, 9000211242301190, 4000001169119001, 9000025619601190, 9000060256401190, 9000134112401190, 9000166163601190, 9000118982801190, 9000207894501190, 9000119600101190, 9000122262501190, 9000143459301190, 9000166166501190, 9000165836601190, 9000047795001190, 9000029567801190, 9000214604601190, 9000149529401190, 9000161385101190, 9000023733901190, 9000060495501190, 9000023017101190, 9000001922701190, 9000001283119001, 9000211564001190, 9000070098401190, 9000062293301190, 9000166178101190, 9000020770101190, 6000001212119001, 9000105278601190, 9000211544201190, 9000118488201190, 9000166181001190, 9000124939401190, 9000115211401190, 9000017342601190, 9000211160201190, 9000002011101190, 9000060711901190, 9000211163101190, 9000106510301190, 9000212315101190, 9000046247901190, 9000001141119001, 9000118494001190, 9000153028401190, 9000166161201190, 9000047044401190, 9000159889201190, 9000214775601190, 9000060254001190, 9000004548401190, 9000211242801190, 9000211166001190, 9000134112901190, 9000187258501190, 9000158205401190, 9000062947801190, 9000042109401190, 9000025623001190, 9000223229401190, 9000107545901190, 9000031155501190, 9000023629101190, 9000188006701190, 9000209894701190, 9000222131501190, 9000184601901190, 9000117888301190, 9000211570201190, 9000042115201190, 9000207542401190, 9000111619201190, 9000034079701190, 9000050463701190, 9000226002901190, 9000119609301190, 9000211567401190, 9000002057001190, 9000166178601190, 9000216431401190, 9000184943401190, 9000211157801190, 9000001931901190, 9000166181501190, 9000048549501190, 9000181592701190, 9000211160701190, 9000118488701190, 9000211544701190, 4000001113119001, 1113119001, 9000164264401190, 9000048808401190, 9000029941201190, 9000135131601190, 9000034065701190, 9000029944101190, 9000137720101190, 9000060254501190, 9000211166501190, 9000120593701190, 9000212318501190, 9000031841401190, 9000187259001190, 9000166190201190, 9000042030201190, 9000185031801190, 9000211553401190, 9000158743501190, 9000054474701190, 9000118474701190, 9000166167501190, 9000209895201190, 9000060493601190, 9000207898401190, 9000205159201190, 9000048797301190, 9000034230901190, 9000038454901190, 4000001284119001, 9000118483401190, 9000166176201190, 9000049286601190, 9000001952201190, 9000180486601190, 9000207036701190, 9000182307101190, 9000023638301190, 9000027452701190, 9000059222301190, 9000211567901190, 9000025635101190, 9000119538801190, 9000130009201190, 9000081830001190, 9000020594801190, 9000018063301190, 9000034959301190, 9000211548101190, 9000213365701190, 9000219458501190, 9000025103301190, 9000163036101190, 9000118236101190, 9000033807301190, 6000001142119001, 9000047096601190, 9000001938201190, 9000042104601190, 9000060255001190, 9000087749401190, 9000166162201190, 9000187256601190, 9000022981401190, 9000211167001190, 9000232491801190, 9000034065201190, 9000062843501190, 9000002145901190, 9000071470701190, 9000166190701190, 9000149755501190, 7000001327119001, 9000118756801190, 9000119601601190, 9000002148801190, 9000025626901190, 9000166173801190, 9000177642601190, 9000047978601190, 9000211562601190, 9000215658601190, 9000166176701190, 9000030112701190, 9000118483901190, 9000156141501190, 9000002003901190, 9000176442301190, 8000001185119001, 9000116387601190, 9000027453201190, 9000020003601190, 9000167843601190, 9000062294801190, 9000211158801190, 9000211235601190, 9000214464101190, 9000080780901190, 9000041638501190, 9000041766501190, 9000185689701190, 9000116646501190, 9000017344101190, 9000117852601190, 9000166185401190, 9000211164601190, 9000225398201190, 9000230902201190, 9000059538701190, 9000074924301190, 9000062021901190, 9000166162701190, 9000060255501190, 9000062841101190, 9000029996301190, 9000030562401190, 9000143970401190, 9000025647201190, 9000119599201190, 9000030152801190, 9000026773601190, 9000060975201190, 9000150191201190, 9000010927201190, 9000002120801190, 9000180683701190, 9000031435701190, 9000074981301190, 9000219928501190, 9000106981301190, 9000114942901190, 9000161128201190, 9000024936201190, 9000161435401190, 9000110389001190, 9000118760201190, 9000223413001190, 9000027857501190, 9000186321501190, 9000023479901190, 9000030417501190, 9000166174301190, 9000015083101190, 9000005611101190, 9000211563101190, 6000001157119001, 9000030497201190, 9000023329201190, 9000059706801190, 9000054689201190, 9000041735601190, 9000037821701190, 9000059709701190, 9000172503301190, 9000002570501190, 9000004644101190, 9000117875801190, 9000015680601190, 9000132826201190, 9000119002201190, 9000166234201190, 9000101056601190, 9000211162201190, 9000181594201190, 9000211165101190, 9000166160301190, 9000023952301190, 9000061942701190, 9000166185901190, 9000017373101190, 9000116647201190, 7000001143119001, 9000060256001190, 9000006675201190, 9000023622401190, 9000166188801190, 9000002044501190, 9000054729301190, 9000002198101190, 9000060975701190, 9000158588501190, 9000109439401190, 9000101605801190, 9000066380201190, 9000166169001190, 9000225689001190, 9000073625001190, 9000220364201190, 9000003125501190, 9000025653501190, 9000166171901190, 9000169679101190, 9000012341501190, 9000060981501190, 9000105118801190, 9000002642001190, 9000088504401190, 9000002181201190, 9000119534501190, 9000054689701190, 9000166177701190, 9000110395301190, 9000145902501190, 9000001672101190, 9000024689401190, 9000166180601190, 9000029937401190, 9000112676601190, 9000002110201190, 9000117233401190, 9000221860601190, 9000212570701190, 9000023873101190, 9000061965901190, 9000101057101190, 9000184052301190, 9000017857101190, 9000211162701190, 9000069264801190, 9000183517601190, 9000164880801190, 9000029354401190, 9000166160801190, 9000211165601190, 9000125536501190, 9000070675701190, 4000001172119001, 9000062947401190, 9000087216201190, 9000030077001190, 9000020579401190, 9000001942601190, 9000027135901190, 9000205849501190, 9000060492701190, 9000166169501190, 9000160281501190, 9000032307101190, 9000207897501190, 7000001101119001, 7000001229119001, 9000166172401190, 9000039631601190, 9000229839601190, 9000090066501190, 9000166175301190, 9000181407301190, 9000104072601190, 9000062011801190, 9000166178201190, 9000119535001190, 9000211157401190, 9000211160301190, 9000118488301190, 9000042225901190, 9000166181101190, 9000154689601190, 9000211547201190, 9000166184001190, 9000034958401190, 9000002036801190, 9000166158401190, 9000060254101190, 9000120593301190, 9000116113301190, 9000001911701190, 9000214775701190, 9000211166101190, 9000166164201190, 9000112020201190, 9000139181801190, 9000028999401190, 9000029946601190, 9000187258601190, 9000051911401190, 9000166189801190, 9000211553001190, 9000164884201190, 9000123133501190, 9000140464701190, 9000028083601190, 9000060493201190, 9000033254801190, 9000215117201190, 9000166170001190, 9000133094801190, 9000209894801190, 9000232141201190, 9000166172901190, 9000196048101190, 9000025654501190, 9000192694501190, 9000021456101190, 9000070096101190, 9000111619301190, 9000136195301190, 9000098025701190, 9000101305401190, 9000157369401190, 9000062521401190, 9000166175801190, 9000119532601190, 9000002028601190, 9000193337401190, 9000211567501190, 9000186812301190, 9000021052301190, 9000041762701190, 9000138428301190, 9000211157901190, 2000001187119001, 9000025637601190, 9000188632801190, 9000211160801190, 9000211570401190, 9000119538401190, 9000166181601190, 9000118488801190, 9000116645601190, 9000029608501190, 9000025333301190, 9000017986101190, 9000211547701190, 9000216206901190, 3000001244119001, 9000212318601190, 9000028254601190, 9000031531401190, 9000060254601190, 9000037905801190, 9000005624201190, 9000033195401190]

In [78]:
clientvisit.iloc[0:10]

,dischargedisposition,dischargelocation,dischargedtm,admitdtm,clientvisitguid,clientguid,MRN
0,Routine,NaN,2018-11-26 18:00:00,2018-11-24 06:00:00,9001267799500270,9000001012800200,193 00 71
1,Routine,NaN,2018-02-08 12:00:00,2018-01-30 05:00:00,9001120183500270,9000001036500200,284 42 27
2,Routine,NaN,2018-11-13 13:00:00,2018-10-21 06:00:00,9001250939200270,9000001036500200,284 42 27
3,Routine,NaN,2018-10-07 19:00:00,2018-09-26 16:00:00,9001238357200270,9000001092600200,205 71 49
4,Routine,NaN,2018-02-02 12:00:00,2018-02-02 03:00:00,9001122045500270,9000002305100200,276 27 33
5,Routine,NaN,2018-11-28 17:00:00,2018-11-25 02:00:00,9001267866400270,9000002082400200,309 90 48
6,Routine,NaN,2018-12-28 17:00:00,2018-12-26 20:00:00,9001283668100270,9000001532200200,275 67 74
7,Routine,NaN,2018-11-16 17:00:00,2018-11-14 21:00:00,9001263930100270,9000008821400200,278 35 36
8,Routine,NaN,2018-12-10 18:00:00,2018-12-04 17:00:00,9001272842600270,9000008648800200,196 94 78
9,Routine,NaN,2018-01-07 14:00:00,2018-01-05 10:00:00,9001107860300270,9000003975000200,244 58 47


In [ ]:
demographics = pd.read_csv(path + 'demographics_2012_2018.txt',sep='\t',names=["clientguid","YOB","language","gender","race","mariatalstatus"])